# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [69]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [70]:
#Import
Weather_data_path = 'Output Data/weather_data.csv'

#Process
Weather_data= pd.read_csv(Weather_data_path)

Weather_data

,Unnamed: 0,City,Longitude,Latitude,Temperature (f),Humidity,Cloudiness,Wind Speed (mph),Country
0,0,Nikolskoye,30.79,59.70,37.40,93,90,8.95,RU
1,1,Blagoveshchenka,79.87,52.83,48.27,49,97,16.53,RU
2,2,Port Alfred,26.89,-33.59,63.54,85,0,12.35,ZA
3,3,Bengkulu,102.27,-3.80,77.11,84,100,2.51,ID
4,4,Palmer,-72.33,42.16,70.00,73,75,5.82,US
...,...,...,...,...,...,...,...,...,...
577,577,Santa Isabel Rio Negro,-65.02,-0.41,83.86,66,70,2.68,BR
578,578,Turan,108.22,16.07,75.20,73,75,2.24,VN
579,579,Dekar,21.93,-21.53,77.43,26,0,18.84,BW
580,580,Batticaloa,81.69,7.71,78.35,76,26,6.33,LK


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [71]:
#Configure and set variables. 
gmaps.configure(api_key=g_key)
locations = Weather_data[['Latitude', 'Longitude']].astype(float)
humidity = Weather_data['Humidity'].astype(float)

In [72]:
#Make a heat map!
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating = False, max_intensity = 100,\
                                point_radius = 1)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [76]:
#My ideal weather... Less than 60 degrees fahrenheidt but more than 30.
#hotel_df = Weather_data[Weather_data['Temperature']]
Notemp = Weather_data[Weather_data['Temperature (f)'] < 60]
#Low humidity because I hate a swamp.
Nohumid = Notemp[Notemp['Humidity'] < 50]
#And finally... No clouds
hotel_df = Nohumid[Nohumid['Cloudiness'] == 0]
hotel_df

,Unnamed: 0,City,Longitude,Latitude,Temperature (f),Humidity,Cloudiness,Wind Speed (mph),Country
190,190,Kumul,93.45,42.80,47.82,48,0,4.05,CN
226,226,Huainan,117.00,32.63,52.84,47,0,6.60,CN
231,231,Wuwei,102.64,37.93,41.00,41,0,3.36,CN
310,310,Rūdbār,62.60,30.15,56.80,30,0,9.44,AF
396,396,Shihezi,86.03,44.30,44.69,49,0,1.43,CN


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [78]:
#New Column for Hotel Names and Countries
hotel_df['Hotel Name'] = ''

#Set Parameters
params = {
    "radius": 5000,
    "types": "hotel",
    'key': g_key}


for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row['Latitude']
    lng = row['Longitude']

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    #print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except:
        print('Data not found')

C:\Users\zachg\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [79]:
hotel_df

,Unnamed: 0,City,Longitude,Latitude,Temperature (f),Humidity,Cloudiness,Wind Speed (mph),Country,Hotel Name
190,190,Kumul,93.45,42.80,47.82,48,0,4.05,CN,Hami
226,226,Huainan,117.00,32.63,52.84,47,0,6.60,CN,Huainan
231,231,Wuwei,102.64,37.93,41.00,41,0,3.36,CN,Wuwei
310,310,Rūdbār,62.60,30.15,56.80,30,0,9.44,AF,Rūdbār
396,396,Shihezi,86.03,44.30,44.69,49,0,1.43,CN,Beiquanzhen


In [87]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]
hotel_layer = gmaps.symbol_layer(locations, fill_color = 'blue', stroke_color = 'black', scale = 2, info_box_content = hotel_info)

In [88]:
# Add marker layer ontop of heat map
fig.add_layer(hotel_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))